# 28. MediaWikiマークアップの除去

27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [ ]:
# 問題28: MediaWikiマークアップの除去

import os
import re

# イギリスの記事ファイルのパス
data_dir = "../data"
uk_article_file = os.path.join(data_dir, "uk_article.txt")

# 基礎情報テンプレートのフィールド名と値を抽出する関数（問題25の改良版）
def extract_basic_info_template(text):
    """記事から基礎情報テンプレートのフィールド名と値を抽出する関数"""
    # 基礎情報テンプレートの抽出
    template_pattern = r"\{\{基礎情報[^|]*?\|([\s\S]*?)\}\}"
    template_match = re.search(template_pattern, text)
    
    if not template_match:
        return {}
    
    template_content = template_match.group(1)
    
    # フィールドの抽出（行ごとに処理）
    basic_info = {}
    field_name = None
    field_value = []
    
    for line in template_content.split("\n"):
        # 新しいフィールドの開始
        field_match = re.match(r"\|\s*([^=]+?)\s*=\s*(.*)", line)
        if field_match:
            # 前のフィールドがあれば保存
            if field_name is not None:
                basic_info[field_name] = "\n".join(field_value).strip()
            
            # 新しいフィールドの開始
            field_name = field_match.group(1).strip()
            field_value = [field_match.group(2).strip()]
        else:
            # 現在のフィールドの値の続き
            if field_name is not None:
                field_value.append(line.strip())
    
    # 最後のフィールドを保存
    if field_name is not None:
        basic_info[field_name] = "\n".join(field_value).strip()
    
    return basic_info

In [ ]:
# マークアップ除去関数群

# 強調マークアップを除去する関数
def remove_emphasis_markup(text):
    """MediaWikiの強調マークアップを除去する関数"""
    # 強い強調かつ斜体（5つのアポストロフィ）
    text = re.sub(r"'''''(.+?)'''''", r"\1", text)
    # 強い強調（3つのアポストロフィ）
    text = re.sub(r"'''(.+?)'''", r"\1", text)
    # 弱い強調（2つのアポストロフィ）
    text = re.sub(r"''(.+?)''", r"\1", text)
    return text

# 内部リンクマークアップを除去する関数
def remove_internal_links(text):
    """MediaWikiの内部リンクマークアップを除去する関数"""
    # パターン1: [[リンク先|表示テキスト]] → 表示テキスト
    text = re.sub(r"\[\[(?:[^|\]]+)\|([^\]]+)\]\]", r"\1", text)
    # パターン2: [[リンク先]] → リンク先
    text = re.sub(r"\[\[([^\]]+)\]\]", r"\1", text)
    return text

# 外部リンクマークアップを除去する関数
def remove_external_links(text):
    """MediaWikiの外部リンクマークアップを除去する関数"""
    # パターン1: [URL 表示テキスト] → 表示テキスト
    text = re.sub(r"\[(?:https?|ftp)://[^\s\]]+\s+([^\]]+)\]", r"\1", text)
    # パターン2: [URL] → URL
    text = re.sub(r"\[((?:https?|ftp)://[^\s\]]+)\]", r"\1", text)
    return text

# HTMLタグを除去する関数
def remove_html_tags(text):
    """HTMLタグを除去する関数"""
    # HTMLタグ: <タグ>内容</タグ> → 内容
    text = re.sub(r"<[^>]+>([^<]*)</[^>]+>", r"\1", text)
    # 単独のHTMLタグ: <タグ /> → 空文字列
    text = re.sub(r"<[^>]+/>", "", text)
    # その他のHTMLタグ: <タグ> → 空文字列
    text = re.sub(r"<[^>]+>", "", text)
    return text

# 参照タグを除去する関数
def remove_ref_tags(text):
    """参照タグを除去する関数"""
    # パターン1: <ref>内容</ref> → 空文字列
    text = re.sub(r"<ref[^>]*>.*?</ref>", "", text)
    # パターン2: <ref name="名前" /> → 空文字列
    text = re.sub(r"<ref[^/>]*/>", "", text)
    return text

# コメントを除去する関数
def remove_comments(text):
    """コメントを除去する関数"""
    # コメント: <!-- コメント --> → 空文字列
    text = re.sub(r"<!--.*?-->", "", text, flags=re.DOTALL)
    return text

# テンプレート呼び出しを除去する関数
def remove_template_calls(text):
    """テンプレート呼び出しを除去する関数"""
    # 単純なテンプレート: {{テンプレート名}} → 空文字列
    text = re.sub(r"\{\{[^|{}]*\}\}", "", text)
    # パラメータ付きテンプレート: {{テンプレート名|パラメータ}} → 空文字列
    text = re.sub(r"\{\{[^{}]*\}\}", "", text)
    return text

In [ ]:
# すべてのマークアップを除去する関数
def remove_all_markup(text):
    """すべてのMediaWikiマークアップを除去する関数"""
    # コメントを除去
    text = remove_comments(text)
    # 参照タグを除去
    text = remove_ref_tags(text)
    # HTMLタグを除去
    text = remove_html_tags(text)
    # 外部リンクマークアップを除去
    text = remove_external_links(text)
    # 内部リンクマークアップを除去
    text = remove_internal_links(text)
    # 強調マークアップを除去
    text = remove_emphasis_markup(text)
    # テンプレート呼び出しを除去
    text = remove_template_calls(text)
    # 連続する空白を1つの空白に置換
    text = re.sub(r"\s+", " ", text)
    # 前後の空白を削除
    text = text.strip()
    return text

# 基礎情報テンプレートからマークアップを除去する関数
def remove_markup_from_template(basic_info):
    """基礎情報テンプレートの値からマークアップを除去する関数"""
    cleaned_info = {}
    for name, value in basic_info.items():
        cleaned_info[name] = remove_all_markup(value)
    return cleaned_info

# メイン処理
try:
    # イギリスの記事を読み込む
    if os.path.exists(uk_article_file):
        with open(uk_article_file, "r", encoding="utf-8") as f:
            uk_article = f.read()
        
        # 基礎情報テンプレートのフィールド名と値を抽出
        basic_info = extract_basic_info_template(uk_article)
        
        # マークアップを除去
        cleaned_info = remove_markup_from_template(basic_info)
        
        # 結果を表示
        print(f"基礎情報テンプレートのフィールド数: {len(cleaned_info)}\n")
        print("マークアップを除去した基礎情報テンプレートの内容:")
        
        # 変更があったフィールドを表示
        for name, value in cleaned_info.items():
            print(f"\n【フィールド名】{name}")
            print(f"【変更前】{basic_info[name][:100]}..." if len(basic_info[name]) > 100 else f"【変更前】{basic_info[name]}")
            print(f"【変更後】{value[:100]}..." if len(value) > 100 else f"【変更後】{value}")
    else:
        print(f"イギリスの記事ファイルが見つかりません: {uk_article_file}")
        print("問題20を先に実行して、イギリスの記事を抽出してください。")
        
except Exception as e:
    print(f"エラーが発生しました: {e}")

## 解説

この問題では、前の問題（問題26と問題27）で実装した強調マークアップと内部リンクマークアップの除去に加えて、その他のMediaWikiマークアップも可能な限り除去する方法を学びます。

### MediaWikiのマークアップの種類

MediaWikiには、以下のようなさまざまなマークアップが存在します：

1. **強調マークアップ**（問題26で対応）
   - 弱い強調（`''テキスト''`）
   - 強い強調（`'''テキスト'''`）
   - 強い強調かつ斜体（`'''''テキスト'''''`）

2. **内部リンクマークアップ**（問題27で対応）
   - 基本形式（`[[リンク先]]`）
   - 表示テキスト指定形式（`[[リンク先|表示テキスト]]`）

3. **外部リンクマークアップ**
   - 基本形式（`[URL]`）
   - 表示テキスト指定形式（`[URL 表示テキスト]`）

4. **HTMLタグ**
   - 開始タグと終了タグのペア（`<タグ>内容</タグ>`）
   - 単独のタグ（`<タグ />`）

5. **参照タグ**
   - 基本形式（`<ref>内容</ref>`）
   - 名前付き参照（`<ref name="名前">内容</ref>`）
   - 参照の再利用（`<ref name="名前" />`）

6. **テンプレート呼び出し**
   - 基本形式（`{{テンプレート名}}`）
   - パラメータ付き（`{{テンプレート名|パラメータ}}`）

### マークアップ除去の実装

これらのマークアップを除去するために、それぞれに対応する関数を実装し、最終的にすべての関数を組み合わせて使用します。処理の順序は重要です。例えば、コメントや参照タグは他のマークアップを含んでいる可能性があるため、先に除去する必要があります。